<a href="https://colab.research.google.com/github/ismeemaahir/portfolio1/blob/master/200820_18%EC%9D%BC%EC%B0%A8_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D(%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 교차검증
- 교차검증의 의미
- 실습

### 방법 1.홀드아웃
- 데이터: 훈련데이터와 테스트데이터
- 훈련데이터: 훈련데이터 + 검증데이터 --> 테스트데이터 

### 방법 2. K폴드 교차검증
- 데이터 수가 적을때 사용
- default 연산량 X K개 만큼
 

- cross_validation_score

In [ ]:
# 교차검증위한 모듈실행

import pandas as pd                         # 데이터프레임 모듈
import matplotlib.pyplot as plt             # 시각화 모듈
import numpy as np                          # 연산 모듈
import seaborn as sns                       # 시각화 모듈
from scipy import stats                     # 수치해석 기능 모듈/stats는 확률분포분석 기능
from sklearn.datasets import load_boston    # 내장데이터 사용(boston 데이터)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
%matplotlib inline
# 실행한 브라우저에서 그림을 바로 볼수 있게 해주는 코드
boston= load_boston()  # 보스턴 데이터 이름 변경

#pandas dataframe 독립변수
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)  # 보스턴 데이터=> 데이터 프레임화하는 작업
                                                                    # 원하는 이름 = pd.DataFrame(바꿔질함수이름.data, columns= 바꿔질함수이름.feature_names)
                                                                 

#종속변수
bostonDF['PRICE'] = boston.target          # 바꿔진데이터프레임 중 하나의 인덱스만 뽑아서 종속변수로 활용
                                           # 바꾼 데이터프레임이름['원하는 인덱스']= 데이터이름(데이터프레임이름아님).target

print('Size of Dataset:', bostonDF.shape)  # 제대로 입력되었는지 확인(info,head,tail,isnull,describe,mean...)
# print(bostonDF)
# print(bostonDF['PRICE'])

Size of Dataset: (506, 14)
        CRIM    ZN  INDUS  CHAS    NOX  ...    TAX  PTRATIO       B  LSTAT  PRICE
0    0.00632  18.0   2.31   0.0  0.538  ...  296.0     15.3  396.90   4.98   24.0
1    0.02731   0.0   7.07   0.0  0.469  ...  242.0     17.8  396.90   9.14   21.6
2    0.02729   0.0   7.07   0.0  0.469  ...  242.0     17.8  392.83   4.03   34.7
3    0.03237   0.0   2.18   0.0  0.458  ...  222.0     18.7  394.63   2.94   33.4
4    0.06905   0.0   2.18   0.0  0.458  ...  222.0     18.7  396.90   5.33   36.2
..       ...   ...    ...   ...    ...  ...    ...      ...     ...    ...    ...
501  0.06263   0.0  11.93   0.0  0.573  ...  273.0     21.0  391.99   9.67   22.4
502  0.04527   0.0  11.93   0.0  0.573  ...  273.0     21.0  396.90   9.08   20.6
503  0.06076   0.0  11.93   0.0  0.573  ...  273.0     21.0  396.90   5.64   23.9
504  0.10959   0.0  11.93   0.0  0.573  ...  273.0     21.0  393.45   6.48   22.0
505  0.04741   0.0  11.93   0.0  0.573  ...  273.0     21.0  396.90   7

### 데이터 탐색
- 시각화, 통계 검정
- 시각화 해볼것
- 변수 정의

## 머신러닝 모형 만들기

In [ ]:
#종속변수를 가져와서 훈련,테스트 과정을 거쳐 예측모형, 정확도 판단

from sklearn.model_selection import train_test_split
# train set(학습데이터셋)과 test set(테스트셋)을 분리하는 모듈 실행
from sklearn.linear_model import LinearRegression
#일반최소제곱 선형회귀: 선형회귀는 선형모델을 학습헤서 데이터세트에서 관찰된 목표와 선형근사에 의해 예측된 목표사이의 잔차제곱합을 최소화 
from sklearn.metrics import mean_squared_error, r2_score
#


#훈련데이터, 테스트 분리
y_target=bostonDF['PRICE']
X_data= bostonDF.drop(['PRICE'], axis=1,inplace= False)
Y_data= bostonDF.drop(['PRICE'], axis=1,inplace= False)

X_train,X_test,y_train,y_test = train_test_split(X_data,y_target,test_size=0.3,random_state=1)

# 모형을 만들기-선형회귀
lr_model =LinearRegression()
lr_model.fit(X_train, y_train)

#예측
y_preds= lr_model.predict(X_test)


- mean_squared_error(MSE): 실제값과 예측값의 차이를 제곱후 평균
- MSE를 RMSE로 변환하는 작업: 실제 오류평균보다 더 커지는 특성이 있어 루트를 씌움


In [ ]:
mse=mean_squared_error(y_test,y_preds)
print(mse)

19.831323672062922


In [ ]:
rmse=np.sqrt(mse)
print(rmse)

4.4532374371981245


## K폴드를 적용
- cross_val_score() 함수 활용해서 RMSE 
- +5 폴드값, 값이

In [ ]:
from sklearn.model_selection import  cross_val_score
neg_mse_scores= cross_val_score(lr_model, X_data,Y_data,scoring="neg_mean_squared_error",cv=5)
print(neg_mse_scores)

[-3.52582905e-27 -2.76764693e-27 -2.62504714e-27 -8.92031840e-27
 -4.83809760e-25]


In [ ]:
rmse_scores=np.sqrt(-1* neg_mse_scores)
print(rmse_scores)

[5.93786919e-14 5.26084302e-14 5.12352139e-14 9.44474372e-14
 6.95564346e-13]


In [ ]:
np.mean(rmse_scores)

1.9064682392371389e-13